# Projekt założenia
symulator zdarzeniowy M/M/1/N
    - średni czas oczekiwania na obsługę
    - prawdopobieństwo blokady
porównać zmodelem analitycznym:
    - dla N = 51
    - różnych wartości obciążenia: 0.01, 0.05, 0.1

średni czas oczekiwania - > histogram


In [32]:
# https://pyshark.com/poisson-distribution-and-poisson-process-in-python/#poisson-distribution-example-in-python
import scipy
import numpy as np
import matplotlib
from enum import Enum
import uuid

czas w serwerze to jest "usługa" 

In [33]:
global_time = 0
n = 51 # max order in a system
# zmienna lososowa opisująca czas między zgłoszeniami
la = 0.9 # intensywność napływu
time_between_packet = 1 / la
# zmienna losowa opisuje czas pomiędzy zgłoszeniami k przez serwer
mu = 1
time_of_service = 1 / mu # intensywność obsługi
time_of_simulation = 10
np.random.seed(12)

In [34]:
roh = la / mu
rn = pow(roh, n + 1)
p0 = (1 - roh) / (1 - rn)         # free queue probability
pb = p0 * rn / roh                

navg = roh / (1-roh) - (n+1) * rn/(1-rn)  
tavg = navg / la                    
u    = mu * (1 - p0)                

out = '''
pb   = %.4f    # block probability
navg = %.4f    # average orders in a system
tavg = %.4f    # average time in a system
u    = %.4f    # throughput
'''
print(out % (pb, navg, tavg, u))

NameError: name 'roh' is not defined

In [ ]:
time_between_packet = 1 / la
# zmienna losowa opisuje czas pomiędzy zgłoszeniami k przez serwer
mu = 1
time_of_service = 1 / mi # intensywność obsługi
time_of_simulation = 10
np.random.seed(12)

In [ ]:
class EventType(Enum):
    IN_EVENT = "IN_EVENT"
    OUT_EVENT = "OUT_EVENT"

# slajd 91, 

class Event():
    def __init__(self, time: float, event_type, client_id = None):
        self.time = time
        self.TEventType = event_type
        if client_id == None:
            self.client_id = str(uuid.uuid4())
        else:
            self.client_id = client_id

In [ ]:
class Server:
    def __init__(self):
        pass

In [ ]:

def put(list, new_event):
    index = -1
    if len(list) + 1 > n: # bufor
        return
    if new_event.time < global_time: # new packet is 
        return

    for (idx, event) in enumerate(list):
        if new_event.time < event.time:
            index = idx
    if index != -1:
        list.insert(index, new_event)


In [ ]:
def get(list):
    if len(list) > 0:
        global_time = list[0]
        return list.pop(0)
    else:
        return None
get(list)
for i in list:
    print(i.time, i.TEventType)

       

4 A


In [ ]:
# https://github.com/gsharma007/Stochastic_OR_and_Simulation_Modeling/blob/master/Queue_Modeling/DES.py

class Simulation:
    def __init__(self):
        self.num_in_system = 0
        self.clock = 0.0
        self.t_arrival = self.generate_interarrival()
        self.t_depart = float('inf') 
        self.num_arrivals = 0
        self.num_departs = 0
        self.total_wait = 0.0
        self.tmp_time = self.generate_service()
        
    def advance_time(self):
        t_event = min(self.t_arrival, self.t_depart)
        self.total_wait += self.num_in_system*(t_event - self.clock)
        self.clock = t_event
        if self.t_arrival <= self.t_depart:
            self.handle_arrival_event()
        else:
            self.handle_depart_event()
    
    def handle_arrival_event(self):
        self.num_in_system += 1
        self.num_arrivals += 1
        if self.num_in_system <= 1:
            self.temp1 = self.generate_service()
            print(">>>>>>>>",self.temp1)
            self.tmp_time=self.temp1
            self.t_depart = self.clock + self.temp1
        self.t_arrival = self.clock + self.generate_interarrival()
        
    def handle_depart_event(self):
        self.num_in_system -= 1
        self.num_departs += 1
        if self.num_in_system > 0:
            self.temp2=self.generate_service();
            print(">>>>><<<<<",self.temp2)
            self.tmp_time = self.temp2
            self.t_depart = self.clock + self.temp2
        else:
            self.t_depart = float("inf")
    
    # zmienieć na poissona

    def generate_interarrival(self):
        return np.random.lognormal(mean=0.6, sigma=0.3)
    def generate_service(self):
        return np.random.uniform(low = 5, high = 8)

    def average_time(self):
        return self.total_wait / s.num_departs


s = Simulation()

clock_time = []
arrival_times = []
service_times = []
departures = []
event_calendar = []


for i in range(5):
    s.advance_time()
    a = s.clock
    b = s.t_arrival
    c = s.tmp_time
    d = s.t_depart
    clock_time.append(a)
    arrival_times.append(b)
    service_times.append(c)
    departures.append(d)
    
    event_list = {"Iteration": "I_"+str(i), "Event_time": a,
                  "Arrival_time": b, "Service_time": c, "Departure_time": d}
    event_calendar.append(event_list)
for interation in event_calendar:
    print(interation)

print(s.average_time()) 

>>>>>>>> 6.601218180140894
>>>>><<<<< 6.656112899793699
{'Iteration': 'I_0', 'Event_time': 2.099912651723645, 'Arrival_time': 3.5851465040463655, 'Service_time': 6.601218180140894, 'Departure_time': 8.701130831864539}
{'Iteration': 'I_1', 'Event_time': 3.5851465040463655, 'Arrival_time': 5.8691797556534455, 'Service_time': 6.601218180140894, 'Departure_time': 8.701130831864539}
{'Iteration': 'I_2', 'Event_time': 5.8691797556534455, 'Arrival_time': 7.018974677838026, 'Service_time': 6.601218180140894, 'Departure_time': 8.701130831864539}
{'Iteration': 'I_3', 'Event_time': 7.018974677838026, 'Arrival_time': 8.843898278376754, 'Service_time': 6.601218180140894, 'Departure_time': 8.701130831864539}
{'Iteration': 'I_4', 'Event_time': 8.701130831864539, 'Arrival_time': 8.843898278376754, 'Service_time': 6.656112899793699, 'Departure_time': 15.357243731658238}
16.231309738196675


get pobiera zdarzenie z listy z namnijeszym czasem, przesuwa czas symulacji do tego czasu